In [1]:
import numpy as np
import cv2

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
# config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.8
session = InteractiveSession(config=config)

from srgan_d import SRGAN
from generator import ImageDataGenerator

Using TensorFlow backend.


In [2]:
# !TF_ENABLE_GPU_GARBAGE_COLLECTION=false

In [3]:
# from tensorflow.python.client import device_lib
# device_lib.list_local_devices()

In [4]:
r = 2
input_dir = f'data/train/{r}/low/'
tgt_dir = f'data/train/{r}/high/'

input_dir_val = f'data/val/{r}/low/'
tgt_dir_val = f'data/val/{r}/high/'


test_file = 'UBMk30rjy0o_17675_42.jpg'
test_low_path =  input_dir_val + test_file
test_high_path = tgt_dir_val + test_file
test_out_path = 'data/out/' + test_file
test_test_path = 'data/test/' + test_file

vgg_path = 'models/vgg16_notop.hdf5'

input_shape = (67, 120, 3)
tgt_shape = (134, 240, 3)

batch_size = 16

In [5]:
srgan = SRGAN(vgg_path, batch_size, input_shape, tgt_shape)
gen = ImageDataGenerator(input_dir, tgt_dir, batch_size)
gen_valid = ImageDataGenerator(input_dir_val, tgt_dir_val, batch_size)

/home/keras/srgan_d.py:110: UserWarning: Update your `Network` call to the Keras 2 API: `Network(name="generator", inputs=Tensor("in..., outputs=Tensor("ou...)`
  self.shared_generator = Network(input=layers[0], output=layers[-1], name='generator')
/home/keras/srgan_d.py:139: UserWarning: Update your `Network` call to the Keras 2 API: `Network(name="discriminator", inputs=Tensor("in..., outputs=Tensor("de...)`
  self.shared_discriminator = Network(input=layers[0], output=layers[-1], name='discriminator')
/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [ ]:
for e in range(10001):
    X_low, X_high = gen.flow_from_directory().__next__()
    srgan.train(X_low, X_high)
    if e % 100 == 0:
        print('epoch', e)
        srgan.valid(X_low, X_high)
        
        test_img_low = (cv2.imread(test_low_path) / 255.0)[np.newaxis, :, :, :]
        test_img_high = (cv2.imread(test_high_path) / 255.0)[np.newaxis, :, :, :]

        test_img_sr = srgan.generator.predict(test_img_low)
        test_img_sr_int = (test_img_sr * 255).astype('u1')[0]
        test_out_path = 'data/out/'  + test_file.split('.')[0] + f'_test_{e}.jpg'
        cv2.imwrite(test_out_path, test_img_sr_int)

epoch 0
d_loss_real:7.424835681915283, d_loss_fake:0.0011580570135265589, vgg_loss:4.483985424041748
epoch 100
d_loss_real:0.6698594093322754, d_loss_fake:0.5411350727081299, vgg_loss:1.1800169944763184
epoch 200
d_loss_real:0.6996190547943115, d_loss_fake:0.6266014575958252, vgg_loss:0.6828950047492981
epoch 300
d_loss_real:0.5520012378692627, d_loss_fake:0.41125160455703735, vgg_loss:0.5830435156822205
epoch 400
d_loss_real:0.5024400949478149, d_loss_fake:0.354085773229599, vgg_loss:0.6755682229995728
epoch 500
d_loss_real:0.6221754550933838, d_loss_fake:0.29660072922706604, vgg_loss:0.6487149596214294
epoch 600
d_loss_real:0.8004252910614014, d_loss_fake:0.6654926538467407, vgg_loss:0.31895315647125244


In [ ]:
srgan.generator.save('models/20200515_generator.h5', include_optimizer=False)